In [1]:
import numpy as np
import pandas as pd
import nupack as nu
import networkx as nx 
import matplotlib.pyplot as plt

from hdna.complex import Complex
from hdna.strand import Strand
from hdna.model import Model
from hdna.chamber import Chamber
from hdna.kinetwork import Kinetwork
from hdna.simulator import Simulator, Options

from pyvis.network import Network

## About Julia-Python interface

In [31]:
import juliacall


jl = juliacall.newmodule('Dude')

jl.seval('using BioSimulator')
jl.seval('using TickTock')
jl.seval('using Base.Threads')

print(jl.nthreads())

jl.tick()
model = jl.Network("AT-all")  

initial = 2

model <= jl.Species("SS",int(initial))
model <= jl.Species("L1",0)
model <= jl.Species("L2",0)
model <= jl.Species("R1",0)
model <= jl.Species("R2",0)
model <= jl.Species("D",0)



kf_dl = 4e8
nonspecific_correction = 1/5

sliding_correction = 1/2
kf_sliding = 1e7 * sliding_correction



Na = 6.023e23

kf_ssl1 = kf_dl*nonspecific_correction
kb_ssl1 = 22.920792678077465 

kf_ssl2 = kf_dl*nonspecific_correction
kb_ssl2 = 0.007808225837362128

kf_ssr2 = kf_dl*nonspecific_correction
kb_ssr2 = 0.0626785751957368

kf_ssr1 = kf_dl*nonspecific_correction
kb_ssr1 = 183.99091641848813

kf_ssD = kf_dl*nonspecific_correction
kb_ssD = 6.902831840829526e-08

kf_l1l2 = kf_sliding
kb_l1l2 = 3406.6124793451313*sliding_correction

kf_l2D = kf_sliding
kb_l2D = 88.40461309148736*sliding_correction

kf_r1r2 = kf_sliding
kb_r1r2 = 3406.6124793451313*sliding_correction

kf_r2D = kf_sliding
kb_r2D = 11.01306438328072*sliding_correction




model <= jl.Reaction("ss-l1_f",  kf_ssl1, "SS + SS --> L1")
model <= jl.Reaction("ss-l1_b",  kb_ssl1, "L1 --> SS + SS")
model <= jl.Reaction("ss-l2_f",  kf_ssl2, "SS + SS --> L2")
model <= jl.Reaction("ss-l2_b",  kb_ssl2, "L2 --> SS + SS")
model <= jl.Reaction("ss-r2_f",  kf_ssr2, "SS + SS --> R2")
model <= jl.Reaction("ss-r2_b",  kb_ssr2, "R2 --> SS + SS")
model <= jl.Reaction("ss-r1_f",  kf_ssr1, "SS + SS --> R1")
model <= jl.Reaction("ss-r1_b",  kb_ssr1, "R1 --> SS + SS")
model <= jl.Reaction("ss-D_f",   kf_ssD, "SS + SS --> D")
model <= jl.Reaction("ss-D_b",   kb_ssD, "D --> SS + SS")
model <= jl.Reaction("l1-l2_f",  kf_l1l2, "L1 --> L2")
model <= jl.Reaction("l1-l2_b",  kb_l1l2, "L2 --> L1")
model <= jl.Reaction("l2-D_f",   kf_l2D, "L2 --> D")
model <= jl.Reaction("l2-D_b",   kb_l2D, "D --> L2")
model <= jl.Reaction("r1-r2_f",  kf_r1r2, "R1 --> R2")
model <= jl.Reaction("r1-r2_b",  kb_r1r2, "R2 --> R1")
model <= jl.Reaction("r2-D_f",   kf_r2D, "R2 --> D")
model <= jl.Reaction("r2-D_b",   kb_r2D, "D --> R2")

model <= jl.Species("oobbooAooddoo", 0)
model <= jl.Reaction("oobbooAooddoo", 0, "D --> oobbooAooddoo")



runtime = 1e-2
Nmonte = 50000
jl.visualize(model)
jl.tock()

jl.tick()
simresults = [jl.simulate(model, jl.Direct(),tfinal=runtime) for i in range(Nmonte)]
results = [{'c':jl.hcat(*simresults[i].u).__array__(), 't':simresults[i].t} for i in range(Nmonte)]
jl.tock()

1


[ Info:  started timer at: 2023-01-13T16:14:05.177


AttributeError: Julia: UndefVarError: visualize not defined

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## About Chamber

In [1]:
import numpy as np
import pandas as pd
import nupack as nu
import networkx as nx 
import matplotlib.pyplot as plt

from hdna.complex import Complex
from hdna.strand import Strand
from hdna.model import Model, Geometry
from hdna.chamber import Chamber
from hdna.kinetwork import Kinetwork, Kinetics
from hdna.simulator import Simulator, Options #it slows down import because it has to wake julia up 

from pyvis.network import Network

In [2]:
model = Model('dna', '3D')
mincore = 3

a = Strand(model, 'AAAAAAAAAAA')
b = Strand(model, 'TTTTTTTTTTT')

kgraph = Kinetwork(model, a, b, mincore)
geometry = Geometry(180, 180)
kinetics = Kinetics(model, kgraph, geometry)
simulator = Simulator(model, kgraph, kinetics)
G = simulator.Graph

In [14]:
# print(simulator.biosim.species_list)
print(simulator.biosim.reaction_list)

OrderedCollections.OrderedDict{Symbol, BioSimulator.Reaction}(:f_sliding_1 => bbbooooooooAdddoooooooo -> bbbboooooooAddddooooooo
  affected by: bbbooooooooAdddoooooooo, bbbboooooooAddddooooooo
  affects: bbbooooooooAdddoooooooo
, :b_sliding_1 => bbbboooooooAddddooooooo -> bbbooooooooAdddoooooooo
  affected by: bbbooooooooAdddoooooooo, bbbboooooooAddddooooooo
  affects: bbbboooooooAddddooooooo
, :f_sliding_2 => bbbboooooooAddddooooooo -> bbbbbooooooAdddddoooooo
  affected by: bbbbbooooooAdddddoooooo, bbbboooooooAddddooooooo
  affects: bbbboooooooAddddooooooo
, :b_sliding_2 => bbbbbooooooAdddddoooooo -> bbbboooooooAddddooooooo
  affected by: bbbbbooooooAdddddoooooo, bbbboooooooAddddooooooo
  affects: bbbbbooooooAdddddoooooo
, :f_sliding_3 => bbbbbooooooAdddddoooooo -> bbbbbboooooAddddddooooo
  affected by: bbbbbooooooAdddddoooooo, bbbbbboooooAddddddooooo
  affects: bbbbbooooooAdddddoooooo
, :b_sliding_3 => bbbbbboooooAddddddooooo -> bbbbbooooooAdddddoooooo
  affected by: bbbbbooooooAdddd

In [22]:
import juliapkg

In [24]:
juliapkg.add('TikzGraphs', uuid='b4f28e30-c73f-5eaf-a395-8a9db949a742')

In [27]:
import juliacall


jl = juliacall.newmodule('Dude')

jl.seval('using BioSimulator')
jl.seval('using TickTock')
jl.seval('using Base.Threads')
jl.seval('using TikzGraphs')

In [30]:
jl.TikzGraphs.Visualize(simulator.biosim)

AttributeError: Julia: UndefVarError: Visualize not defined